In [ ]:
from astroquery.sdss import SDSS
import astropy.units as u

In [ ]:
import ipyaladin as ipyal
aladin= ipyal.Aladin(target='Berkeley 20', fov=0.42, survey='P/SDSS9/color')
aladin

In [ ]:
from astroquery.sdss import SDSS
query = """
SELECT TOP 3200
       p.objID,
       p.ra,
       p.dec,
       p.u,
       p.g,
       p.r,
       p.i,
       p.z
FROM PhotoPrimary AS p
JOIN dbo.fGetNearbyObjEq(83.15416667, 0.18833333, 3.24) AS r ON r.objID = p.objID
WHERE p.clean = 1 and p.probPSF = 1
"""
cluster_cat = SDSS.query_sql(query)

In [ ]:
query_apo = """
SELECT TOP 5600 photoobj.objID as objID, photoobj.run, photoobj.camcol, photoobj.field, photoobj.obj,
photoobj.psfmag_u AS u, photoobj.psfmag_g AS g, photoobj.psfmag_r AS r, photoobj.psfmag_i AS i,
photoobj.psfmag_z AS z, photoobj.ra as photoobj_ra, photoobj.dec as photoobj_dec,
photoobj.ra AS ra, photoobj.dec AS dec, star.apstar_id, star.ra as star_ra, star.dec as star_dec,
aspcap.teff, aspcap.param_m_h, aspcap.logg
FROM apogeeStar AS star
JOIN fGetNearbyApogeeStarEq(250.423458,36.461306,60) AS near ON star.apstar_id = near.apstar_id
CROSS APPLY dbo.fGetNearestObjEq(star.ra, star.dec, 3.6) AS near_s
JOIN photoobj ON near_s.objid=photoobj.objid
JOIN aspcapStar as aspcap ON star.apstar_id = aspcap.apstar_id
"""
cluster_apo_cat = SDSS.query_sql(query_apo)

In [ ]:
len(cluster_apo_cat)
cluster_apo_cat

In [ ]:
aladin.add_table(cluster_cat)

In [ ]:
import astropixie
import astropixie_widgets as aww

In [ ]:
aww.data.SDSSRegion??

In [ ]:
sdss_reg = astropixie.data.SDSSRegion(cluster_cat)
sdss_reg.to_array()
teffs = aww.science.teff(sdss_reg)
lums = aww.science.luminosity(sdss_reg)
lums

In [ ]:
# new with SDSS data:
aww.config.setup_notebook()
fig = aww.visual.hr_diagram_figure(sdss_reg)
aww.visual.show(aww.visual.hr_diagram_figure(sdss_reg))

In [ ]:
# versus the original data:
aww.visual.hr_diagram('berkeley20')